# Tests on dinov2 embeddings

## Imports

In [3]:
import os
import torch

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Checking torch cache dir
Warning : make sure to set torch hub cache dir to reuse downloade

In [4]:
torch.hub.get_dir()

'/linkhome/idris/genidr/ssos023/.cache/torch/hub'

## Setting torch cache dir for download

In [5]:
# Assuming $ALL_CCFRWORK is an environment variable that stores the desired path
hub_dir = os.path.join(os.getenv('ALL_CCFRWORK'),'models')

if hub_dir is not None:
    torch.hub.set_dir(hub_dir)
else:
    print("Environment variable 'ALL_CCFRWORK' is not set.")


## Download dinov2 weights

In [ ]:
# DINOv2 with registers
dinov2_vits14_reg = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14_reg')
dinov2_vitb14_reg = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14_reg')
dinov2_vitl14_reg = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14_reg')
#dinov2_vitg14_reg = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14_reg')

## Loading a dinov2

In [23]:
dinov2_vits14_reg = torch.hub.load(repo_or_dir='facebookresearch/dinov2', model='dinov2_vits14_reg').to(device)

Using cache found in /gpfswork/rech/ads/commun/models/facebookresearch_dinov2_main


In [24]:
random_image = torch.randn(1, 3, 224, 224).to(device)  # Batch size of 1

output = dinov2_vits14_reg(random_image)

In [26]:
output.shape

torch.Size([1, 384])